In [1]:
# Extract the data from the Mauritius harvested files to compatible format for the visualizer

In [2]:
import os,sys
import pandas as pd
import numpy as np 
import h5py as h5
from tqdm import tqdm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/tmp/ipykernel_2513284/2179053797.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [3]:
# inter_files = [data+'/'+f for f in os.listdir('/data') if 'intercomparison' in f ]

### document
https://wmoomm.sharepoint.com/sites/wmocpdb/eve_activityarea/Forms/AllItems.aspx?id=%2Fsites%2Fwmocpdb%2Feve%5Factivityarea%2FInstruments%20and%20Methods%20of%20Observation%20Programme%20%28IMOP%29%5F67452102%2D7575%2De911%2Da98e%2D000d3a44bd9c%2FIntercomparisons%2FPast%20Intercomparisons%2FRSO%2DIC%2D2005%5FFinal%5FReport%2Epdf&parent=%2Fsites%2Fwmocpdb%2Feve%5Factivityarea%2FInstruments%20and%20Methods%20of%20Observation%20Programme%20%28IMOP%29%5F67452102%2D7575%2De911%2Da98e%2D000d3a44bd9c%2FIntercomparisons%2FPast%20Intercomparisons&p=true&ga=1


In [4]:
digitized = '/scratch/das/federico/databases_service2/MAURITIUS/'


In [5]:
### create a single file for each sonde type

# extracting the list of different sensor types
sonde_types = np.unique( [ f.split('_')[1].split('.csv')[0] for f in os.listdir(digitized+'/temp/') if '.csv' in f  ])
sonde_types

array(['Graw', 'Graw-GPS', 'MKII', 'Meisei', 'Modem', 'SRS', 'Sip',
       'Vaisala', 'Vaisala-GPS'], dtype='<U11')

In [6]:
# creating single files out of the single ascent files, per sensor 

# output directory for the processed files 
outdir = 'data_processed'
if not os.path.isdir(outdir):
    os.mkdir(outdir)
    

In [7]:
# creating single files out of the single ascent files, per sensor 

# output directory for the processed files 
outdir = 'data_processed'
if not os.path.isdir(outdir):
    os.mkdir(outdir)

if not os.path.isdir(outdir+'/single_ascent'):
        os.mkdir(outdir+'/single_ascent')

#sonde_types = ['Vaisala']

# looping over each distinct sensor, all the ascents
for s in sonde_types:

    ### temperature
    print('+++ Extracting temperature data +++ ')
    all_df_temp = []
    files_temp = [ digitized + '/temp/' + f for f in os.listdir(digitized+'/temp/') if f.split('_')[1].split('.csv')[0]== s ]  
    files_temp.sort()
    for f in tqdm(files_temp):
        df_temp = pd.read_csv(f , sep = ',' )
        df_temp = df_temp[ [c for c in df_temp.columns if c in ['press','datetime','temp','hum']] ]
        all_df_temp.append(df_temp)

    df_sum_temp = pd.concat(all_df_temp)
    df_sum_temp = df_sum_temp.round(decimals=2)
    df_sum_temp['datetime']  = [v[:19] for v in df_sum_temp['datetime'] ]
    df_sum_temp['datetime'] = pd.to_datetime(df_sum_temp.datetime, format="%d-%m-%Y %H:%M:%S" ,  utc=True)
    df_sum_temp['datetime'] = pd.to_datetime(df_sum_temp.datetime,  utc=True)
    df_sum_temp = df_sum_temp.dropna(subset=['press'])
    df_sum_temp['press'] = df_sum_temp.press.astype(int)
    df_sum_temp['sensor'] = s 
    df_sum_temp.to_csv( outdir + '/' + s + '_all_ascents_temp.csv' , sep ='\t' )

    ### humidity    
    print('+++ Extracting humidity data +++ ')
    all_df_hum = []
    files_hum = [ digitized + '/hum/' + f for f in os.listdir(digitized+'/hum/') if f.split('_')[1].split('.csv')[0]== s ]
    files_hum.sort()
    for f in tqdm(files_hum):
        df_hum = pd.read_csv(f , sep = ',')
        df_hum = df_hum[ [c for c in df_hum.columns if c in ['press','datetime','temp','hum']] ]
        all_df_hum.append(df_hum)

    df_sum_hum = pd.concat(all_df_hum)
    df_sum_hum = df_sum_hum.round(decimals=2)
    df_sum_hum['datetime']  = [v[:19] for v in df_sum_hum['datetime'] ]
    df_sum_hum['datetime'] = pd.to_datetime(df_sum_hum.datetime, format="%d-%m-%Y %H:%M:%S" ,  utc=True)    
    df_sum_hum = df_sum_hum.dropna(subset=['press'])
    df_sum_hum['press'] = df_sum_hum.press.astype(int)
    df_sum_hum['sensor'] = s
    df_sum_hum.to_csv( outdir + '/' + s + '_all_ascents_hum.csv' , sep ='\t' )


    # Extracting all time stamps
    all_times = list( np.unique(df_sum_temp.datetime) ) + list( np.unique(df_sum_temp.datetime) ) 
    
    # merging temp and hum data in one single file per sonde 
    all_df = []
    for dt in all_times:
        hum = df_sum_hum.loc[df_sum_hum['datetime'] == dt ][['press','hum']]
        temp = df_sum_temp.loc[df_sum_temp['datetime'] == dt ][['press','temp']]

        mer= pd.merge(hum,temp, on='press', how='outer')

        #mer = pd.concat( [ hum.join(temp, on='press'), temp.join(hum, on='press') ]  ).drop_duplicates(subset=['press'] )
        #mer = mer[['press','hum','temp']]
        mer['datetime'] = dt 
        all_df.append(mer)


    df = pd.concat(all_df)
    df = df.sort_values(by=['datetime','press']) 
    df['sensor'] = s

    
    df.to_csv( outdir + '/' + s + '_all_ascents_fulldata.csv' , sep ='\t' )
    red = df.iloc[::5, :]
    red.to_csv( outdir + '/' + s + '_all_ascents_reduced_data.csv' , sep ='\t' )

    
    # Extracting all date_times values 
    if s =='Vaisala':
        dates,times = [],[] 
        for t in np.unique(df_sum_temp['datetime']):
            times.append(t.time())
            dates.append(t.date())
            #print(t.date(), t.time())

        df = pd.DataFrame.from_dict( {'date': dates , 'time': times } )
        
        df.to_csv('all_timestamps.csv' , sep = '\t') 

+++ Extracting temperature data +++ 


100%|██████████| 29/29 [00:00<00:00, 161.87it/s]


+++ Extracting humidity data +++ 


100%|██████████| 31/31 [00:00<00:00, 133.29it/s]


+++ Extracting temperature data +++ 


...

100%|██████████| 61/61 [00:03<00:00, 16.77it/s]


+++ Extracting temperature data +++ 


100%|██████████| 61/61 [00:01<00:00, 39.23it/s]


+++ Extracting humidity data +++ 


100%|██████████| 61/61 [00:03<00:00, 18.18it/s]


In [8]:
all_times = list( np.unique(df_sum_temp.datetime) ) + list( np.unique(df_sum_hum.datetime) ) 

In [9]:
all_times[:2]

[Timestamp('2005-02-07 10:01:53+0000', tz='UTC'),
 Timestamp('2005-02-07 14:57:50+0000', tz='UTC')]

In [10]:
# Dividing into launch files (all sensors included)

if not os.path.isdir(outdir+'/single_ascent/'):
    os.mkdir(outdir+'/single_ascent/')
    
for t in all_times:
    all_data = []
    files = [outdir+'/'+f for f in os.listdir(outdir) if 'fulldata' in f  ]
    print(t)
    for fi in files:
        df = pd.read_csv(fi, sep ='\t' )
        #print('original   ' , df.head() )
        dff = df.loc[ df.datetime == str(t) ]
        #print('datetime loc ' ,  dff.head() ) 
        all_data.append(dff)
    DF = pd.concat(all_data)
    #print( 'DF ' , DF.head() )
    DF.to_csv( outdir+'/single_ascent/' + str(t)[:19] + '.csv' , sep='\t')
    df = DF.iloc[::5, :]
    df.to_csv( outdir+'/single_ascent/' + str(t)[:19] + '_reduced.csv' , sep='\t')
    

2005-02-07 10:01:53+00:00
2005-02-07 14:57:50+00:00
2005-02-07 19:31:41+00:00
2005-02-08 05:13:16+00:00
2005-02-08 10:00:56+00:00
2005-02-08 15:03:03+00:00
2005-02-08 19:33:03+00:00
2005-02-09 05:12:00+00:00
2005-02-09 10:05:05+00:00
2005-02-09 14:59:48+00:00
2005-02-09 19:17:51+00:00
2005-02-10 05:02:13+00:00
2005-02-10 09:58:41+00:00
2005-02-10 14:57:54+00:00
2005-02-10 19:00:19+00:00
2005-02-11 05:02:45+00:00
2005-02-11 10:44:04+00:00
2005-02-11 15:05:21+00:00
2005-02-11 18:31:58+00:00
2005-02-12 05:02:39+00:00
2005-02-12 15:00:05+00:00
2005-02-12 18:36:55+00:00
2005-02-14 05:06:57+00:00
2005-02-14 10:08:04+00:00
2005-02-14 15:00:54+00:00
2005-02-14 18:00:36+00:00
2005-02-15 05:16:46+00:00
2005-02-15 10:03:06+00:00
2005-02-15 15:01:34+00:00
2005-02-15 18:11:25+00:00
2005-02-16 05:11:08+00:00
2005-02-16 10:03:34+00:00
2005-02-16 15:07:48+00:00
2005-02-16 18:11:15+00:00
2005-02-17 10:15:24+00:00
2005-02-17 14:59:11+00:00
2005-02-17 18:03:18+00:00
2005-02-18 05:13:26+00:00
2005-02-18 1

In [11]:
len(df_sum_temp)
df_sum_temp.head(5)

,press,temp,datetime,sensor
0,431,300.48,2005-02-07 10:01:53+00:00,Vaisala-GPS
1,435,298.65,2005-02-07 10:01:53+00:00,Vaisala-GPS
2,439,300.80,2005-02-07 10:01:53+00:00,Vaisala-GPS
3,443,299.38,2005-02-07 10:01:53+00:00,Vaisala-GPS
4,448,298.38,2005-02-07 10:01:53+00:00,Vaisala-GPS


In [12]:
df_sum_temp.datetime

all_times = list( np.unique(df_sum_temp.datetime) ) + list( np.unique(df_sum_hum.datetime) ) 


In [13]:
#len(df_sum_hum)
#df_sum_hum.head(5)

In [14]:
all_df = []
for dt in all_times:
    hum = df_sum_hum.loc[df_sum_hum['datetime'] == dt ]
    temp = df_sum_temp.loc[df_sum_temp['datetime'] == dt ]

    mer= pd.merge(a,b, on='press', how='outer')
    mer = mer[['press','hum','temp']]
    mer['datetime'] = dt 
    all_df.append(mer)




NameError: name 'a' is not defined